### [Store the files in the world_criminality bucket]

In [7]:
from google.cloud import storage 
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/quentinmahieu/Desktop/School/Monash/Projects_Monash/13.ETL_project/credentials/crimeproject-295908-6ab56d225adb.json"

In [8]:
# If you don't specify credentials when constructing the client, the
# client library will look for credentials in the environment.
client = storage.Client()
# Make an authenticated API request
buckets = list(client.list_buckets())
print(buckets)
bucket_name = buckets[0].name

[<Bucket: world_criminality>]


In [10]:
files = os.listdir('Data')
for file in files:
    file_name = file
    path = 'Data/' + file_name
    client.get_bucket(bucket_name).blob(file_name).\
        upload_from_filename(path, content_type='text/csv')

### [How to query the world_criminality bucket]

In [50]:
import pandas as pd
client = storage.Client()
#list the available buckets 
buckets = list(client.list_buckets())
bucket_name = buckets[0].name
bucket = client.get_bucket(bucket_name)
#list the available blobs/objects
blobs = bucket.list_blobs()
     
dfs = {}
for blob in blobs:
    file_name = blob.name
    df = pd.read_csv(f'gs://{bucket_name}/{file_name}',index_col=0)
    dfs[file_name.split('.')[0]] = df
    print(file_name.split('.')[0])

country_gdp
country_info
country_names
crime_rate
happiness
prison_pop


In [51]:
country_gdp = dfs['country_gdp']
country_info = dfs['country_info']
country_names = dfs['country_names']
crime_rate = dfs['crime_rate']
happiness = dfs['happiness']
prison_pop = dfs['prison_pop']

In [52]:
crime_happiness_df = crime_rate.merge(happiness,on='country_code',how='left')

In [55]:
crime_happiness_df

,country_x,crime_index,country_code,country_y,rank,score,GDP,social_support,life_expectancy,freedom,trust_governement,generosity,year
0,Venezuela,84.49,VE,Venezuela,23.0,6.810,1.044240,1.255960,0.720520,0.429080,0.110690,0.058410,2015.0
1,Venezuela,84.49,VE,Venezuela,44.0,6.084,1.133670,1.033020,0.619040,0.198470,0.083040,0.042500,2016.0
2,Venezuela,84.49,VE,Venezuela,82.0,5.250,1.128431,1.431338,0.617144,0.153997,0.065020,0.064491,2017.0
3,Venezuela,84.49,VE,Venezuela,102.0,4.806,0.996000,1.469000,0.657000,0.133000,0.052000,0.056000,2018.0
4,Venezuela,84.49,VE,Venezuela,108.0,4.707,0.960000,1.427000,0.805000,0.154000,0.047000,0.064000,2019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
689,Qatar,11.86,QA,Qatar,28.0,6.611,1.690420,1.078600,0.797330,0.640400,0.522080,0.325730,2015.0
690,Qatar,11.86,QA,Qatar,36.0,6.375,1.824270,0.879640,0.717230,0.566790,0.480490,0.323880,2016.0
691,Qatar,11.86,QA,Qatar,35.0,6.375,1.870766,1.274297,0.710098,0.604131,0.330474,0.439299,2017.0
692,Qatar,11.86,QA,Qatar,32.0,6.374,1.649000,1.303000,0.748000,0.654000,0.171000,0.256000,2018.0
